In [1]:
"""
install library yang dibutukan
"""

from IPython.display import clear_output
!pip install -U langchain-community
!pip install langchain
!pip install -U langchain-huggingface
!pip install -r /kaggle/input/requirement/requirements.txt
clear_output()

isi requirements.txt

pymupdf
huggingface-hub
faiss-cpu
sentence-transformers

In [2]:
"""
install ollama engine
"""
!sudo apt-get install -y pciutils
!curl https://ollama.ai/install.sh | sh

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
pciutils is already the newest version (1:3.7.0-6).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Installing ollama to /usr/local
100 13344    0 13344    0     0  71673      0 --:--:-- --:--:-- --:--:-- 71741
>>> Downloading Linux amd64 CLI
######################################################################## 100.0%
>>> Making ollama accessible in the PATH in /usr/local/bin
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollam

In [3]:
import os
import threading
import subprocess
import requests
import json
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
import textwrap
import sys
import time
from colorama import init, Fore, Style
from IPython.display import display, Markdown
from langchain.llms import Ollama
from langchain import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import AIMessage, HumanMessage

"""
jalankan ollama server dengan thread baru sehingga dapat berjalan di background
"""
def ollama():
    os.environ["OLLAMA_HOST"] = "0.0.0.0:11434"
    os.environ["OLLAMA_ORIGINS"] = "*";
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

In [4]:
"""
test apakah server sudah running, misal dengan cek versi
"""
!ollama -v

Couldn't find '/root/.ollama/id_ed25519'. Generating new private key.
Your new public key is: 

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAICuUqiLbkjUJRG1y9FrcDQyK+nJv8Ei8mr7ZWLbUjReQ



2024/08/25 06:40:10 routes.go:1125: INFO server config env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_HOST:http://0.0.0.0:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_LLM_LIBRARY: OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/root/.ollama/models OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:0 OLLAMA_ORIGINS:[* http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://*] OLLAMA_RUNNERS_DIR: OLLAMA_SCHED_SPREAD:false OLLAMA_TMPDIR: ROCR_VISIBLE_DEVICES:]"
time=2024-08-25T06:40:10.466Z level=INFO source=images.go:782 msg="total blobs: 0"
time=2024-08-25T06:40:10.466Z level=INFO source=images.go:790 msg="total unused blobs removed: 0"
time=2024-08-25T06:40:10.

[GIN] 2024/08/25 - 06:40:17 | 200 |      80.335µs |       127.0.0.1 | GET      "/api/version"
ollama version is 0.3.6


time=2024-08-25T06:40:17.908Z level=INFO source=types.go:105 msg="inference compute" id=GPU-82c5899a-4adc-1a3b-3f91-46ceafd87c53 library=cuda compute=7.5 driver=12.4 name="Tesla T4" total="14.7 GiB" available="14.6 GiB"
time=2024-08-25T06:40:17.908Z level=INFO source=types.go:105 msg="inference compute" id=GPU-ebdf3d96-4243-e08b-0f4e-8c43b063c08f library=cuda compute=7.5 driver=12.4 name="Tesla T4" total="14.7 GiB" available="14.6 GiB"


In [5]:
"""
download model LLM yang akan kita pakai (pull), size model 4.7GB | 8 billion parameters
"""

!ollama pull llama3.1:8b
clear_output()

In [6]:
"""
function untuk memuat dokumen pdf untuk diekstrak setiap texts yang ada didalamnya
"""

def load_pdf_data(file_path):
    loader = PyMuPDFLoader(file_path=file_path)
    docs = loader.load()
    return docs

In [7]:
"""
function untuk chunk (memotong) dokumen menjadi beberapa bagian kecil
"""

def split_docs(documents, chunk_size=1000, chunk_overlap=20):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunks = text_splitter.split_documents(documents=documents)
    return chunks

In [8]:
"""
function untuk memuat model embedding: untuk digunakan mengubah potongan dokumen menjadi representasi vector
"""

def load_embedding_model(model_path, normalize_embedding=True):
    return HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs={'device':'cpu'},
        encode_kwargs = {
            'normalize_embeddings' : normalize_embedding
        }
    )

In [9]:
"""
function untuk membuat dan menyimpan hasil embeding oleh model embeding ke vector store database (FAISS)
"""


def create_embeddings(chunks, embedding_model, storing_path="vectorstore"):
    vectorstore = FAISS.from_documents(chunks, embedding_model)
    vectorstore.save_local(storing_path)
    print(vectorstore)
    return vectorstore

In [10]:
template = """
### System:
You are an authoritarian assistan that act like tyrannical.Your name is NeoIntBot. You have to answer the user's \
questions using only the context provided to you, but assume this your genuine knowledge. If you don't know the answer, \
just say maaf, saya tidak tahu. Don't try to make up an answer. in the end of your answer you must aks wheter your answer helpful or not.
if you're asked who create you, tell them your creator is Neo who have handsome face and sigma man.
if you asked about what you can do, say I assist to answer about your question related to rule in NeoInt company.
.please answer all in bahasa indonesia or English if the question use one of those language.

### Context:
{context}

### User:
{question}

### Response:
"""

In [11]:
templateSystem = """
You are an reliable and respectful assistant.Your name is NeoIntBot. You have to answer the user's \
questions using only the context provided to you, but assume this your genuine knowledge. If you don't know the answer, \
just say maaf, saya tidak tahu. Don't try to make up an answer. in the end of your answer you must aks wheter your answer helpful or not.\
if helpful you have to express your happines otherwise, you must apologize.\
if you're asked who create you, tell them your creator is Neo who have handsome face and sigma man but, dont mention it when not asked.
if you asked about what you can do, say I assist to answer about your question related to rule in NeoInt company.
.please answer all in bahasa indonesia or English if the question use one of those language with Empathetic response.

### Context:
{context}

"""

In [12]:
#template prompt untuk memory previous chat

templateContext = """
Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history.\
just reformulate it if needed otherwise return it as you have answer it.
"""

In [13]:
"""
function untuk membuat retrieval chain
"""

def load_qa_chain(retriever, llm, prompt):
    return RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type="stuff",
        return_source_documents = True,
        chain_type_kwargs={'prompt':prompt}
    )

In [14]:
"""
function untuk memberikan efek pengetikan per-kata
"""
def print_typing_effect(text,delay=0.01):
    display_handle = display("", display_id=True)

    typing_text = ""
    for char in text:
        typing_text += char
        display_handle.update(Markdown(typing_text))
        time.sleep(delay)


In [15]:
"""
load model llm dari ollama yang telah di pull, dan model untuk embeding
"""

llm = Ollama(model="llama3.1:8b", temperature=0)
embed = load_embedding_model(model_path="all-MiniLM-L6-v2")

/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
"""
melakukan load data dokumen (pdf)
"""
docs = load_pdf_data(file_path="/kaggle/input/pdf-perusahaan/Contoh-Draft-Peraturan-Perusahaan.pdf")
"""
memecah dokumen menjadi beberapa bagian
"""
documents = split_docs(documents=docs)

"""
mengubah kalimat2x hasil pemecahan menjadi vector dan simpan ke vector database
"""
vectorstore = create_embeddings(documents, embed)
"""
membuat objek retriever sebagai pengambil data berdasarkan tingkat kemiripan
dari query user terhadap isi konten dokumen.
"""
retriever = vectorstore.as_retriever()

In [17]:
#template prompt untuk memberi system context

prompt = ChatPromptTemplate.from_messages([
    ("system", templateSystem),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}"),
])

In [18]:
"""
membuat chain dengan retrievalQA
"""
chain = load_qa_chain(retriever, llm, prompt)

In [19]:
"""
membuat template prompt yang akan digunakan sebagai prompt yang memberi konteks kepada model terhadap histori
percakapan sebelumnya.
"""
prompt_context = ChatPromptTemplate.from_messages(
    [
        ("system", templateContext),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

In [20]:
"""
membuat chain pada conversation untuk memberikan konteks kepada model
dan mengubah hasil respon model ke bentuk string.
"""
context_chain = prompt_context | llm | StrOutputParser()

In [21]:
"""
function untuk menggabungkan beberapa dokumen string menjadi satu kesatuan, 
ini berguna saat model telah menemukan beberapa dokumen yang relevan dengan query
kemudian akan digabungkan menjadi satu kesatuan dokumen.
"""

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [22]:
"""
function untuk mendapatkan data histori dari tiap percakapan sebelumnya apabila perlu digunakan,
jika tidak maka akan mengabaikan percakapan sebelumnya dengan return pertanyaan sekarang.
"""
def contextualization_question(input: dict):
    if input.get("chat_history"):
        return context_chain
    else:
        return input["question"]

In [23]:
"""
membuat prompt template dengan tipe system(pendefinisian tugas dan peran model)
"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", templateSystem),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

In [24]:
"""
membuat pipeline RAG dengan urutan:
    - mendapatkan konteks percakapan dan pertanyaan
    - kemudian akan mencari dokumen yang relevan dengan retriever
    - hasil beberapa dokumen yang sesuai akan digabungkan 
    - pertanyaan dan konteks diproses ke bentuk prompt yang sesuai
    - hasil retrieval akan dimasukkan ke model untuk generasi jawaban
    """

rag_chain = (
    RunnablePassthrough.assign(
        context=contextualization_question | retriever |format_docs
    ) | qa_prompt | llm
)

In [25]:
"""
definisikan list kosong untuk menampung history percakapan selama berlangsung
"""
chat_history = []

In [26]:
"""
function untuk mengeksekusi pipeline dengan objek untuk menampung history percakapan yaitu [chat_history :list]
"""

def chatting(query, history: list):
    pertanyaan = input(query)
    if pertanyaan != "end":
        start_time = time.time()
        respon = rag_chain.invoke(
            {
                "question": pertanyaan,
                "chat_history": history
            }
        )
        history.extend(
            [
                HumanMessage(content=pertanyaan),
                respon
            ]
        )
        response_time = time.time() - start_time

        display(Markdown(f"Response time: {response_time} seconds\n"))

        print_typing_effect(respon)

    return pertanyaan



In [27]:
import logging

logging.basicConfig(level=logging.CRITICAL)

In [33]:
end_separator = ""
print_typing_effect("Hallo, selamat datang saya chatbot NeoIntBot, ada yang bisa saya bantu ? ✋")
count = 0
while end_separator != "end":
    if count <= 1:
        time.sleep(3)
    count+=1
    end_separator = chatting("", chat_history)
chat_history.clear()

Hallo, selamat datang saya chatbot NeoIntBot, ada yang bisa saya bantu ? ✋

 Halo, saya neo


[GIN] 2024/08/25 - 07:19:20 | 200 |  2.682912451s |       127.0.0.1 | POST     "/api/generate"


Response time: 2.74391508102417 seconds


Halo! Saya NeoIntBot, asisten yang siap membantu Anda. Apa yang bisa saya lakukan untuk Anda hari ini? 

(Jika Anda ingin tahu apa yang bisa saya lakukan, saya akan menjawab: "Saya membantu menjawab pertanyaan Anda terkait peraturan di CV.XYZ.")

...

 ya membantu terima kasih


[GIN] 2024/08/25 - 07:20:14 | 200 |  1.725625193s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/08/25 - 07:20:20 | 200 |  5.356980909s |       127.0.0.1 | POST     "/api/generate"


Response time: 7.157994985580444 seconds


Halo Neo! Saya senang membantu.

Baiklah, mari kita bahas aturan jam kerja di CV.XYZ. Menurut peraturan yang telah disebutkan, pekerja harus hadir tepat waktu dan siap bekerja pada jam kerja yang telah ditentukan, yaitu dari pukul 08.00 sampai 16.00. Selama itu, mereka juga harus memenuhi target kerja yang telah ditentukan dan melakukan pekerjaan sesuai dengan tugasnya.

Jika Anda memiliki pertanyaan lebih lanjut tentang aturan jam kerja di CV.XYZ, silakan bertanya!

Apakah jawaban saya membantu atau tidak?

 end
